# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

## Statistics

In [ ]:
import os

from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.preprocessing.preprocessor import make_original_dataframe, make_resized_dataframe, make_noised_dataframe
from debruiteur.noise.noise import GaussianNoise, PoissonNoise, UniformNoise, SaltPepperNoise, SquareMaskNoise, SpeckleNoise
from debruiteur.utils.utils import load_model, split_train_val_df
from debruiteur.noise.filters import gaussian_filter, wiener_filter, laplacian_filter, gaussian_weighted_substract_filter
from debruiteur.noise.filters import mean_filter, median_filter, conservative_filter, low_pass_filter, high_pass_filter
from debruiteur.statistics.statistics import compute_noise_reduction_method_statistics, compute_noise_type_statistics
from debruiteur.metrics.metrics import metrics_example

In [ ]:
import debruiteur
dir(debruiteur.noise.filters)

In [ ]:
noise_class_list = [
    GaussianNoise(mean=0, std=20),
    PoissonNoise(),
    UniformNoise(amplitude=100),
    SaltPepperNoise(),
    SquareMaskNoise(mask_shape=(10, 10), freq=0.1),
    SpeckleNoise(),
]

### Load data

In [ ]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, "images"))
df_resized = make_resized_dataframe(df_original, img_shape=(100, 100), resized_path=os.path.join(working_dir, "resized_images"))
df_noised = make_noised_dataframe(df_resized, noise_class_list, os.path.join(working_dir, "noised_images"))

### Metrics

Mean squared error (MSE) :
$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Normalized root mean squared error (NRMSE) :
$$\text{NRMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(\log{\frac{\hat{y} + 1}{y + 1}}\right)^2}$$

Peak signal to noise ration (PSNR) :
$$\text{PSNR} = 10 \log_{10} \left( \frac{{MAX_{I}^2}}{MSE} \right)$$

Structural similarity (SSIM) :
$$\text{SSIM(x, y)} = \frac{(2\mu_x \mu_y + c_1)(2\sigma_{xy} + c_2)}{(\mu_x^2 + \mu_y^2 + c_1)(\sigma_x^2 + \sigma_y^2 + c_2)}$$

### Show metrics example

In [ ]:
metrics_example(df_noised, noise_class_list)

You can see the score for each type of noise for each metric. We can notice that the type of noise that deteriorates the image the most is SpeckleNoise. We can also see that the Gaussian, Poisson noise is one of the filters that deteriorates the image the least. 

### Create data generator

In [ ]:
stats_gen = DataGenerator(df_noised[0:20], batch_size=20)

### Load models

In [ ]:
conv_ae_model = load_model(os.path.join(working_dir, "saved_models"), "conv_autoencoder.h5")
dense_ae_model = load_model(os.path.join(working_dir, "saved_models"), "dense_autoencoder.h5")
gan_model = load_model(os.path.join(working_dir, "saved_models"), "gan_generator.h5")

### Noise reduction methods
After the output of the neural networks we apply a filter to smooth the image. Indeed, the images at the output of the neural network are a little blurred. These filters allow to reduce the blur. The filters used are : 
* Wiener filter
* Laplacian filter
* Gaussian Weigthed filter

In [ ]:

noise_reduction_methods = [('Gaussian Filter', lambda img: gaussian_filter(img * 255)),
                           ('Mean Filter', lambda img: mean_filter(img * 255)),
                           ('Median Filter', lambda img: median_filter(img * 255)),
                           ('Conservative Filter', lambda img: conservative_filter(img * 255)),
                           ('Low pass FFT Filter', lambda img: low_pass_filter(img * 255)),
                           
                           ('Convolutional Autoencoder None', lambda x : conv_ae_model.predict(x.reshape(1, 100, 100, 1)) * 255),
                           ('Convolutional Autoencoder Wiener', lambda x : wiener_filter(conv_ae_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255)),
                           ('Convolutional Autoencoder Laplacian', lambda x : laplacian_filter(conv_ae_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255)),
                           ('Convolutional Autoencoder Gaussian Weighted', lambda x : gaussian_weighted_substract_filter(conv_ae_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255)),

                           ('Dense Autoencoder', lambda x : dense_ae_model.predict(x.reshape(1, 10000)) * 255),

                           ('Generative Adversarial Network None', lambda x : gan_model.predict(x.reshape(1, 100, 100, 1)) * 255),
                           ('Generative Adversarial Network Wiener', lambda x : wiener_filter(gan_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255)),
                           ('Generative Adversarial Network Laplacian', lambda x : laplacian_filter(gan_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255)),
                           ('Generative Adversarial Network Weighted', lambda x : gaussian_weighted_substract_filter(gan_model.predict(x.reshape(1, 100, 100, 1)).reshape(100,100) * 255))]

### Noise reduction methods metrics averaged on all kind of noises

In [ ]:
compute_noise_reduction_method_statistics(stats_gen, noise_reduction_methods)

### Noise reductions methods for each noise type
#### Structural similarity (SSIM)

A high SSIM is desired

In [ ]:
df_stat_ssim = compute_noise_type_statistics(stats_gen, noise_reduction_methods, noise_class_list, "SSIM")
df_stat_ssim

In [ ]:
df_stat_ssim.mean(axis=1).sort_values()

#### Peak signal-to-noise ratio (PSNR)

A High PSNR is desired but not in all cases. It is often used as a reconstruction loss when compressing images.

In [ ]:
df_stat_psnr = compute_noise_type_statistics(stats_gen, noise_reduction_methods, noise_class_list, "PSNR")
df_stat_psnr

In [ ]:
df_stat_psnr.mean(axis=1).sort_values()

#### Mean squared error (MSE)

A low MSE is desired.

In [ ]:
df_stat_mse = compute_noise_type_statistics(stats_gen, noise_reduction_methods, noise_class_list, "MSE")
df_stat_mse

In [ ]:
df_stat_mse.mean(axis=1).sort_values()

#### Normalized root mean squared error (NRMSE)

A low NRMSE is desired.

In [ ]:
df_stat_nrmse = compute_noise_type_statistics(stats_gen, noise_reduction_methods, noise_class_list, "NRMSE")
df_stat_nrmse

In [ ]:
df_stat_nrmse.mean(axis=1).sort_values()

# Conclusion

As we can observe the conservative filter has the best score on all most all type of noises.  
It may be because the changes in the images are very small so it the denoised image is similar to the original image.  
This is particularly true when the images are not very noised and the performance would be different if the images had more noise.  

The generative adversial neural network reduces the noise quite well, but as the image is generated, the colors are sometimes different which is not the case of other filters.  
When we sum all these small differences pixelwise it is expected to have a high error.  
Even though the results are visually good.  

For each filter the metrics where computed on all kind of noises.  
Usually we should use a specific filter on a particular noise as they perform better on the approriate noise.  

Comparing the neural networks, the generative adversial neural network has the best performances.  
If we had a GPU with more RAM, we could increase the depth of the GAN by putting more convolutional blocks.  
This is also the reason why we reduced the image's size to 100x100 and used grayscale.  
An other improvement would be to implement the style loss in order to keep better texture details and so improve the performance.  

Another point to note is that the structural similarity has been criticized for not being more corelated to the human's perception as the mean squarred error.  
Comparing the images visually is sometimes more valuable than comparing metrics.  

This conclusion shouldn't be used to say that classical filters (gaussian etc) perform always better than neural networks in all cases.  